In [1]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm
from models import vgg

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import numpy as np
def manual_seed(seed):
    np.random.seed(seed) #1
    random.seed(seed) #2
    torch.manual_seed(seed) #3
    torch.cuda.manual_seed(seed) #4.1
    torch.cuda.manual_seed_all(seed) #4.2
    torch.backends.cudnn.benchmark = False #5 
    torch.backends.cudnn.deterministic = True #6

manual_seed(42)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = vgg.load_vgg16()
model.classifier[6] = nn.Linear(in_features=4096, out_features=100, bias=True)
# freeze convolution weights
# for param in model.features.parameters():
#     param.requires_grad = False

for name, param  in model.classifier[6].named_parameters():
    print(name, param.shape)

cuda:0
weight torch.Size([100, 4096])
bias torch.Size([100])


In [4]:
EPOCHS = 20
BATCH = 32
VAL_BATCH=256
LR = 1e-3
MOMENTUM = 0.9

In [5]:
transform = transforms.Compose([
    # transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5071, 0.4867, 0.4408), std= (0.2675, 0.2565, 0.2761)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5071, 0.4867, 0.4408), std= (0.2675, 0.2565, 0.2761)),
])

train_data = torchvision.datasets.CIFAR100(root="./dataset", train=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH,
                                          shuffle=True)
val_data = torchvision.datasets.CIFAR100(root="./dataset", train=False, transform=test_transform)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=VAL_BATCH,
                                          shuffle=False)

print(next(iter(val_loader))[0].shape)

torch.Size([256, 3, 32, 32])


In [6]:
# optimizer
optimizer = optim.SGD(model.classifier.parameters(), lr=LR, momentum=MOMENTUM)
# loss function
criterion = nn.CrossEntropyLoss()

In [7]:
model.to(device)

def validate(model, test_loader):
    model.eval()
    val_loss = 0.0
    val_acc = 0
    with torch.no_grad():
        for data in tqdm(test_loader,leave=True):
            imgs, target = data[0].to(device), data[1].to(device)
            output = model(imgs)
            loss = criterion(output, target)

            val_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_acc += (preds==target).sum().item()
    
    val_loss = val_loss/len(test_loader)
    val_acc = 100. * val_acc/len(test_loader.dataset)

    return val_loss, val_acc


start_loss, start_acc = validate(model, val_loader)
print(f"START LOSS {start_loss:.4f}, ACC {start_acc:.2f}")
best = 0
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    train_acc = 0
    for data in tqdm(train_loader,leave=True):
        imgs, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(imgs)
        loss = criterion(output, target)
        _, preds = torch.max(output.data, 1)

        train_acc += (preds ==target).sum().item()
        loss.backward()
        optimizer.step()

    train_loss = train_loss/len(train_loader)
    train_acc = 100. * train_acc/len(train_loader.dataset)
    print(f" TRAIN_LOSS : {train_loss:.4f}, TRAIN_ACC : {train_acc:.2f}")

    val_loss, val_acc = validate(model, val_loader)
    print(f" VAL_LOSS : {val_loss:.4f}, VAL_ACC : {val_acc:.2f}")
    if best < val_acc:
        best = val_acc
        checkpoint = {
            'model' : model,
            'model_state_dict' : model.state_dict(),
        }
        torch.save(checkpoint, './checkpoint/vgg16.pth')
        print(f"save best acc {best:.2f}")


  0%|          | 0/40 [00:10<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x25088 and 512x4096)

In [ ]:
filepath = './checkpoint/vgg16.pth'
checkpoint = torch.load(filepath)
model = checkpoint['model']
print(model)
model.load_state_dict(checkpoint['model_state_dict'])

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

<All keys matched successfully>